使用keras框架生成最基本的GAN，首先复习一下作为神经网络界的 "hello world",基础数据集 MNIST(Modified National Institute of Standards and Technology)

大量的机器学习框架中，默认自带这样的数据库，用于对开发模型进行标签训练和测试

常规的MNIST测试集中包含了 60000组训练图片与10000组测试图片

- 黑白图像,图片尺寸 (28,28,1)
- 隐含编码维度100维
- 定义生成器函数
- 定义判别器函数
- 定义训练函数

UpSampling2D

图片数据在高与宽的方向进行数据插值倍增

[B,H,W,C] -> size(a,b) = [B,H×a,W×b,C]

In [25]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, ZeroPadding2D, LeakyReLU, UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import numpy as np

<img src="../img/08.jpg" style="zoom:67%;" />
<caption><center> 生成器</caption>

In [ ]:
def build_generator(self):
    """
        搭建生成器：
        用上采样+卷积代替池化层
        中间不包含全连接
        加入BN
        激活函数使用ReLU，输出层使用 Tanh
        """
    model = Sequential()
    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
    model.add(Reshape((7, 7, 128)))

    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
    model.add(Activation="tanh")

    model.summary()
    noise = Input(shape=(self.latent_dim, ))
    img = model(noise)

    return Model(noise, img)


<img src="../img/09.jpg" style="zoom:67%;" />
<caption><center> 判别器</caption>

In [ ]:
def build_discriminator(self):
    """
        搭建判别器
        步长为为2的卷积层代替池化层
        中间不包含全连接层
        使用 BN
        激活函数用 LeakyReLU，斜率0.2
        """
    model = Sequential()
    model.add(
        Conv2D(32,
               kernel_size=3,
               strides=2,
               input_shape=self.img_shape,
               padding="same"))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernal_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=-0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, Activation='sigmoid'))
    model.summary()
    img = Input(shape=self.img_shape)
    validity = model(img)

    return Model(img, validity)


In [ ]:
def train(self, epochs, batch_size=128, save_interval=50):
    """
        从MNIST载入数据
        将输入数据缩放到 [-1,1]范围内
        训练过程:先用生成数据与真实数据训练判别器，用随机输入和训练好的判别器训练生成器 
        """
    # 加载数据
    (X_train, _), (_, _) = mnist.load_data()
    # 尺度变换
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        d_loss_real = self.discriminiator.train_on_batch(imgs, valid)
        d_loss_fake = self.discriminiator.train_on_batch(gen_imgs, fake)

        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        # 训练生成器
        g_loss = self.combined.train_on_batch(noise, valid)
        # 打印 d_loss和g_loss
        print("D loss: %s, G loss: %s" % (d_loss, g_loss))


In [ ]:
help(BatchNormalization)

In [26]:
# GAN的实现
class DCGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        # 隐含编码维度100维
        self.latent_dim = 100
        # 优化器设置，使用的为 Adam，学习率为 0.0002,动量 β1-0.5
        optimizer = Adam(0.0002, 0.5)
        # 构建并编译判别器
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
        # 构建生成器
        self.generator = self.build_generator()
        # 生成器接收输入为随机噪音，并输出图片
        z = Input(shape=(100,))
        img = self.generator(z)
        # 判别器输入的是图片，并判断是否有效
        self.discriminator.trainable = False
        valid = self.discriminator(img)
        # 整合两个模型
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):
        """
        搭建生成器：
        用上采样+卷积代替池化层
        中间不包含全连接
        加入BN
        激活函数使用ReLU，输出层使用 Tanh
        """
        model = Sequential()
        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))

        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation="tanh")

        model.summary()
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):
        """
        搭建判别器
        步长为为2的卷积层代替池化层
        中间不包含全连接层
        使用 BN
        激活函数用 LeakyReLU，斜率0.2
        """
        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))

        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1), (0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernal_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, Activation='sigmoid'))
        model.summary()
        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, save_interval=50):
        """
        从MNIST载入数据
        将输入数据缩放到 [-1,1]范围内
        训练过程:先用生成数据与真实数据训练判别器，用随机输入和训练好的判别器训练生成器 
        """
        # 加载数据
        (X_train, _), (_, _) = mnist.load_data()
        # 尺度变换
        X_train = X_train / 127.5 -1.
        X_train = np.expand_dims(X_train, axis=3)
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        for epoch in range(epochs):
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            # 训练生成器
            g_loss = self.combined.train_on_batch(noise, valid)
            # 打印 d_loss和g_loss
            print("D loss: %s, G loss: %s"%(d_loss, g_loss))
